In [8]:
import tensorflow as tf
(train_set, train_labels), (test_set, test_labels) = tf.keras.datasets.cifar100.load_data()
#pixels goes from 255 to 0, so lets normalize it

train_set, test_set = train_set / 255.0, test_set / 255.0

In [11]:
import numpy as np

[np.unique(train_labels), train_set.shape]

[array([ 0,  1,  2,  3,  4,  5,  6,  7,  8,  9, 10, 11, 12, 13, 14, 15, 16,
        17, 18, 19, 20, 21, 22, 23, 24, 25, 26, 27, 28, 29, 30, 31, 32, 33,
        34, 35, 36, 37, 38, 39, 40, 41, 42, 43, 44, 45, 46, 47, 48, 49, 50,
        51, 52, 53, 54, 55, 56, 57, 58, 59, 60, 61, 62, 63, 64, 65, 66, 67,
        68, 69, 70, 71, 72, 73, 74, 75, 76, 77, 78, 79, 80, 81, 82, 83, 84,
        85, 86, 87, 88, 89, 90, 91, 92, 93, 94, 95, 96, 97, 98, 99]),
 (50000, 32, 32, 3)]

In [24]:
class ImageClassifier(tf.keras.Model):
    def __init__(self, input_shape, class_num, dense_units):
        super(ImageClassifier, self).__init__()
        #the filter input is the depth of the spatial convolutional layer 
        self.conv_one = tf.keras.layers.Conv2D(filters=32, kernel_size=(3,3), padding='same',activation='relu', input_shape=input_shape )
        self.pooler_one = tf.keras.layers.MaxPool2D(pool_size=(2,2))
        self.conv_two = tf.keras.layers.Conv2D(filters=64, kernel_size=(3,3), activation='relu' )
        self.pooler_two = tf.keras.layers.MaxPool2D(pool_size=(2,2))
        self.conv_two = tf.keras.layers.Conv2D(filters=64, kernel_size=(3,3), activation='relu' )
        self.flatten = tf.keras.layers.Flatten()
        self.dense = tf.keras.layers.Dense(activation='relu', units=dense_units)
        self.outputer = tf.keras.layers.Dense(units=class_num)
    
    def call(self, inputs):
        feature_map = self.conv_one(inputs)
        reduced_feature_map = self.pooler_one(feature_map)
        feature_map = self.conv_two(reduced_feature_map)
        reduced_feature_map = self.pooler_two(feature_map)
        y = self.dense(self.flatten(reduced_feature_map))
        return self.outputer(y)

In [25]:
model = ImageClassifier(input_shape=train_set[1:], class_num=len(np.unique(train_labels)), dense_units=64)
model.compile(optimizer='adam', loss=tf.keras.losses.SparseCategoricalCrossentropy(from_logits=True))

In [26]:
model(tf.constant([train_set[0]]))

<tf.Tensor: shape=(1, 100), dtype=float32, numpy=
array([[ 1.40391499e-01, -3.60426046e-02, -1.33843720e-02,
         1.47753879e-01, -1.38495699e-01, -4.97373454e-02,
        -2.22868949e-01, -1.98249996e-01,  5.43484185e-03,
        -1.43263444e-01,  2.45363370e-01, -7.91144893e-02,
         2.52974052e-02,  1.29834741e-01,  1.29390270e-01,
        -2.30635479e-02, -1.72334030e-01, -4.97252159e-02,
        -1.12760574e-01, -1.05333894e-01,  1.07235678e-01,
         1.08760200e-01, -6.87817186e-02,  1.90408334e-01,
         1.55926317e-01, -2.46524382e-02,  1.03838123e-01,
        -2.37877835e-02, -1.93362743e-01,  3.66759114e-02,
        -9.59209874e-02, -4.97182459e-02, -4.15482484e-02,
         8.86428729e-02,  1.59696981e-01,  7.93603882e-02,
         5.81055060e-02,  6.60125911e-03,  2.23186806e-01,
        -1.68699175e-01, -1.15351602e-01, -3.10631722e-01,
         2.41522640e-01,  3.84535342e-02,  1.25762492e-01,
         2.72864401e-02, -9.57488418e-02,  1.76042840e-01,
      

In [27]:
model.fit(train_set,train_labels, 
          epochs=20,
          validation_data=(test_set, test_labels))

Epoch 1/10
 191/1563 [==>...........................] - ETA: 49s - loss: 4.4733

KeyboardInterrupt: 

In [ ]:
model.evaluate((test_set,test_labels))